In [30]:
import pandas as pd
import  random
import schedule
import time
from math import floor
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
from datetime import datetime, date, timedelta
from faker import Faker

fake = Faker()

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP" #"HIST_ATUALIZACOES"
driver = "ODBC Driver 17 for SQL Server"

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring, pool_size=10, max_overflow=30)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print(f"Query teste executada com sucesso, conexão com o banco de dados {bancodados} bem-sucedida")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
testar_conexao_sqlalchemy(engine) 

lista_idproduto_bancodados = (pd.read_sql('SELECT IDPRODUTO FROM PRODUTO', engine)['IDPRODUTO']).to_list()
lista_idfornecedor_bancodados = (pd.read_sql('SELECT IDFORNECEDOR FROM FORNECEDOR', engine)['IDFORNECEDOR']).to_list()

lista_idproduto = []
lista_idfornecedor = []

i = 0

# Loop while para processar cada valor de lista1
while i < len(lista_idproduto_bancodados):
    valor_idfornevedor = lista_idproduto_bancodados[i]
    
    # Seleciona n valores aleatórios de lista2
    valores_aleatorios = random.sample(lista_idfornecedor_bancodados, random.randint(1, 10))
    
    # Adiciona os valores ao resultado
    for valor in valores_aleatorios:
        lista_idproduto.append(valor_idfornevedor)
        lista_idfornecedor.append(valor)
    
    # Incrementa o contador
    i += 1   

df_item_prod_fornecedor = pd.DataFrame({
                        'ID_PRODUTO':    lista_idproduto,
                        'ID_FORNECEDOR': lista_idfornecedor,

})


tipo_dados_item_prod_fornecedor = {
                        'ID_PRODUTO':    INTEGER,
                        'ID_FORNECEDOR': INTEGER,

}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE ITEM_PROD_FORNECEDOR"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

try:
    df_item_prod_fornecedor.to_sql(name='ITEM_PROD_FORNECEDOR', con=engine,if_exists='append',index=False,dtype=tipo_dados_item_prod_fornecedor)
    print(f"Dados adicionadas ao banco de dados com sucesso.")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")


#display(df_item_prod_fornecedor)



Query teste executada com sucesso, conexão com o banco de dados MERCADO_OLTP bem-sucedida
Dados adicionadas ao banco de dados com sucesso.
